In [ ]:
from pyspark.sql.functions import *

# Create a Spark DataFrame with 10 columns
df = spark.createDataFrame([
  (1, "John", "Doe", 25, "M", "NY", "USA", "123456789", "john.doe@example.com", "active"),
  (2, "Jane", "Doe", 30, "F", "CA", "USA", "987654321", "jane.doe@example.com", "inactive"),
  (3, "Jim", "Smith", 35, "M", "TX", "USA", "121212121", "jim.smith@example.com", "active"),
  (4, "Joan", "Smith", 40, "F", "FL", "USA", "343434343", "joan.smith@example.com", "inactive"),
  (5, "Jake", "Johnson", 45, "M", "AZ", "USA", "565656565", "jake.johnson@example.com", "active")
], ["id", "first_name", "last_name", "age", "gender", "state", "country", "zipcode", "email", "status"])

# Write the DataFrame as a Delta table with schema evolution enabled
df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/customers")

# Read the Delta table
delta_table = spark.read.format("delta").load("/mnt/delta/customers")

# Create a new DataFrame with only 4 columns
df2 = spark.createDataFrame([
  (6, "John", "Smith", 50),
  (7, "Jane", "Johnson", 55),
  (8, "Jim", "Williams", 60),
  (9, "Joan", "Jones", 65),
  (10, "Jake", "Brown", 70)
], ["id", "first_name", "last_name", "age"])

# Append the new DataFrame to the Delta table
df2.write.format("delta").mode("append").save("/mnt/delta/customers")

# Read the updated Delta table
delta_table = spark.read.format("delta").load("/mnt/delta/customers")

# Verify the schema evolution
delta_table.printSchema()


In [ ]:
# Create a Spark DataFrame with 10 columns
df = spark.createDataFrame([
  (1, "John", "Doe", 25, "M", "NY", "USA", "123456789", "john.doe@example.com", "active"),
  (2, "Jane", "Doe", 30, "F", "CA", "USA", "987654321", "jane.doe@example.com", "inactive"),
  (3, "Jim", "Smith", 35, "M", "TX", "USA", "121212121", "jim.smith@example.com", "active"),
  (4, "Joan", "Smith", 40, "F", "FL", "USA", "343434343", "joan.smith@example.com", "inactive"),
  (5, "Jake", "Johnson", 45, "M", "AZ", "USA", "565656565", "jake.johnson@example.com", "active")
], ["id", "first_name", "last_name", "age", "gender", "state", "country", "zipcode", "email", "status"])

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("customers")

# Write the DataFrame as a Delta table with schema evolution enabled
spark.sql("""
  CREATE TABLE customers_delta
  USING delta
  OPTIONS ('mergeSchema' 'true')
  AS SELECT * FROM customers
""")

# Create a new DataFrame with only 4 columns
df2 = spark.createDataFrame([
  (6, "John", "Smith", 50),
  (7, "Jane", "Johnson", 55),
  (8, "Jim", "Williams", 60),
  (9, "Joan", "Jones", 65),
  (10, "Jake", "Brown", 70)
], ["id", "first_name", "last_name", "age"])

# Create a temporary view of the new DataFrame
df2.createOrReplaceTempView("customers2")

# Append the new DataFrame to the Delta table
spark.sql("""
  INSERT INTO customers_delta
  SELECT * FROM customers2
""")

# Read the updated Delta table
delta_table = spark.read.format("delta").load("/mnt/delta/customers_delta")

# Verify the schema evolution
delta_table.printSchema()